In [110]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="temp",
    passwd="password",
    database ="ecommerce"
    
)

mycursor = mydb.cursor()

In [12]:
class fill_tables:

  def __init__(self,mydb):
    self.cursor=mydb.cursor()
    self.mydb = mydb

  def insert_into_user_table(self,user_id,user_name,email,pasw,city,state,country,phone,doj,toj):

    query= """
    INSERT INTO Users VALUES(%s,%s,
    %s,%s,%s,
    %s,%s,%s, %s,%s);
    """

    self.cursor.execute(query,(user_id,user_name,email,pasw,city,state,country,phone,doj,toj))
    self.mydb.commit()


  def insert_into_retailer_table(self,user_name,email,city,state,country):

    query= """
      INSERT INTO Retailer VALUES(%s,%s,%s,%s,%s);

      """

    self.cursor.execute(query,(user_name,email,city,state,country))
    self.mydb.commit()

  def insert_into_delivery_address(self,user_ID,city,country,india,phone,pin):

    query= """
    INSERT INTO Delivery_Address(user_ID,city,state,country,Phone,Pin) VALUES(%s,%s,%s,%s,%s,%s);
    """

    self.cursor.execute(query,(user_ID,city,country,india,phone,pin))
    self.mydb.commit()

  def insert_into_order_table(self,product_ID,user_ID,order_Date,order_time,address_ID,quantity,rating,review):

    query= """
    INSERT INTO Orders(product_ID,user_ID,order_Date,order_Time,address_ID,quantity,rating,review) VALUES(%s,%s,%s,%s,%s,%s,%s,%s);

    """

    self.cursor.execute(query,(product_ID,user_ID,order_Date,order_time,address_ID,quantity,rating,review))
    self.mydb.commit()

  def insert_into_history_table(self,product_ID,user_ID,No_of_visites):

    query="""
    INSERT INTO History(user_ID,product_ID,No_of_visits) 
    VALUES(%s,%s,%s);
    """

    self.cursor.execute(query,(product_ID,user_ID,No_of_visites))
    self.mydb.commit()

  
  def insert_into_cart_table(self,user_ID,product_ID,quantity):
      query = """
       INSERT INTO Cart(user_ID,product_ID,quantity) 
       VALUES(%s,%s,%s);
      """

      self.cursor.execute(query,(user_ID,product_ID,quantity))
      self.mydb.commit()




  def insert_into_product_details(self,tablename,product_name,category_id,date_added,rating,price,retailer_id,quantity_stock):

      
      query="""
      INSERT INTO %s VALUES(%s,%s,%s,%s,%s,%s,%s);
      """

      self.cursor.execute(query,(tablename,product_name,category_id,date_added,rating,price,retailer_id,quantity_stock))
      self.mydb.commit()




In [14]:
insert = fill_tables(mydb)
insert.insert_into_user_table('10000','shamir','shamir@gmail.com','passw','Ahmedabad','Gujarat','India','1234567890','2021-02-23','2330')


In [17]:
query = "select * from Users where user_ID=10000"
mycursor.execute(query)

for line in mycursor:
    print(line)

(10000, 'shamir', 'shamir@gmail.com', 'passw', 'Ahmedabad', 'Gujarat', 'India', '1234567890', datetime.date(2021, 2, 23), datetime.timedelta(seconds=1410))


### 2. Delete a user from the database. After deleting the user update name of the user as ‘Anonymous’ in all the ratings and reviews written by that user.

In [44]:


def delete_from_user(mydb,user_ID):

    query0="""
    SET foreign_key_checks = 0;
    """
    query1="""
    update Orders set user_ID=88888 where user_ID = %s;"""

    query2 = """
    delete from Users where user_ID= %s;"""

    query3="""
    SET foreign_key_checks = 1;
    """
    mycursor= mydb.cursor()
    mycursor.execute(query0)
    mycursor.execute(query1,(user_ID,))
    mycursor.execute(query2,(user_ID,))
    mycursor.execute(query3)
    mydb.commit()

In [45]:
delete_from_user(mydb,'4')

### 3. Increment the price of all products priced below Rs. 5000 by 10%, which were viewed by more than 10 users in the last 3 months. 


In [51]:

def increase_price(mydb,price_limit,interval,view_limit,percentage_increase):

    query="""

        update Products
        inner join 
        (select B.product_ID,B.price,count(B.date_visited) as Views
        from
        (select A.product_ID,A.date_visited,A.price
        from
        (select Products.product_ID, Products.price,
        History.user_ID,History.date_visited from
        History inner join Products
        on Products.product_ID =History.product_ID
        and Products.price<%s) as A
        where A.date_visited> now()- INTERVAL %s month) as B
        group by B.product_ID) as C
        on Products.product_ID=C.product_ID
        and C.Views>%s
        set Products.price=%s*Products.price;
        """
    test_query="""
    select C.product_ID,C.price,C.Views from
    Products inner join
    (select B.product_ID,B.price,count(B.date_visited) as Views
        from
        (select A.product_ID,A.date_visited,A.price
        from
        (select Products.product_ID, Products.price,
        History.user_ID,History.date_visited from
        History inner join Products
        on Products.product_ID =History.product_ID
        and Products.price<%s) as A
        where A.date_visited> now()- INTERVAL %s month) as B
        group by B.product_ID) as C
        on Products.product_ID=C.product_ID
        and C.Views>%s
    """
    percentage_increase = 1+ percentage_increase/100
    mycursor=mydb.cursor()
    mycursor.execute(test_query,(price_limit,interval,view_limit))

    print("Price Before")
    print("=======================")
    print("Product_ID | Price | Views")
    

    print("--------------------------")
    

    for line in mycursor:
        print(line)
    mycursor.execute(query,(price_limit,interval,view_limit,percentage_increase))
    mycursor.execute(test_query,(price_limit,interval,view_limit))
    
    print("\n\nPrice After")
    print("=======================")

    print("Product_ID | Price | Views")

    print("--------------------------")
    for line in mycursor:
        print(line)
    mydb.commit()




In [52]:
increase_price(mydb,5000,3,10,10)


Price Before
Product_ID | Price | Views
--------------------------
(27, 1549.23, 20)
(52, 1223.79, 20)


Price After
Product_ID | Price | Views
--------------------------
(27, 1704.15, 20)
(52, 1346.17, 20)


### 5. Find phone numbers and email IDs of all users who reside in the city ‘Madrid’ and have made a total purchase greater than or equal to Rs. 10000 in the past.

In [70]:
def find_data(mydb,city,total_purchase):

    query="""
    select C.user_name,C.email,C.phone from 
    (select B.user_name,B.email,B.phone,
    SUM(B.price) total_purchase from
    (select A.*,Products.price from
    (select Users.user_ID,Users.user_name,Users.email,
    Users.phone,Orders.product_ID 
    from Users inner join Orders
    on Users.user_ID = Orders.user_ID
    and Users.city=%s) as A
    inner join Products
    on A.product_ID = Products.product_ID) as B
    group by B.user_name,B.email,B.phone) as C
    where C.total_purchase>%s;
    """
    dbcursor=mydb.cursor()
    dbcursor.execute(query,(city,total_purchase))
    
    for line in dbcursor:
      print(line)

In [74]:
find_data(mydb,'Madrid',10000)

('Carl', 'Carl@gmail.com', '1666898865')
('Carla', 'Carla@gmail.com', '2658844727')


### 6. Find all products in the database whose name contains the string ‘mi’. E.g. Xiaomi, etc, and all users who bought them at least once.

In [63]:
def find_data(mydb,word):
    query ="""
    select Name,product_ID 
    from Electronics 
    where 
    Name LIKE %s union all 
    select Name,product_ID 
    from Novels 
    where Name LIKE %s union all 
    select Name,product_ID 
    from Clothes where Name LIKE %s ;
    """

    word = "%"+word+"%"
    dbcursor =mydb.cursor()
    dbcursor.execute(query,(word,word,word))
    
    for line in dbcursor:
        print (line)


In [64]:
find_data(mydb,'mi')

('Xiaomi Redmi 8', 2)
('Xiaomi Redmi 9', 3)
('Xiaomi Redmi Note 10 Pro', 4)
('Mi Notebook Horizon', 20)
('Cosmic Byte Star Dust', 33)


### 13. Sort all laptops according to the price in increasing order. 

In [97]:
  def sort_electronics(mydb,sub_category):
    
    query = """
    select Products.product_Name,Products.price 
    from Products,Electronics
    where (Products.product_ID = Electronics.product_ID 
    and Electronics.sub_category=%s) 
    ORDER BY price DESC;

    """
    dbcursor=mydb.cursor()
    dbcursor.execute(query,(sub_category,))
    for line in dbcursor:
        print(line)

In [98]:
sort_electronics(mydb,'Laptop')

('Apple Macbook Pro', 122000.0)
('HP Omen V2341', 75660.0)
('Lenovo Ideapad S1234', 59990.0)
('Apple Macbook Air', 55600.0)
('Mi Notebook Horizon', 54999.0)
('Asus Vivobook Ultra K14', 42500.0)
('Dell Vostro H3424', 38400.0)
('HP Pavillion', 35600.0)
('Dell Inspiron 3452', 34999.0)
('Acer Aspire', 33300.0)
('Lenovo Thinkpad HV324', 31500.0)
('Acer One 14 Z2-485', 25990.0)


### 16. Print the UserId, mobile number, and Email Id of all users who have saved a product in the cart, whose quantity is less than 5. 

In [105]:
 def list_product_cart(mydb,quantity):
    
    query = """
    select Cart.user_ID,Users.phone,
    Users.email
    from Cart,Users where
    Cart.quantity<%s
    and Users.user_ID=Cart.user_ID;

    """
    dbcursor=mydb.cursor()
    dbcursor.execute(query,(quantity,))
    for line in dbcursor:
        print(line)

In [106]:
list_product_cart(mydb,5)

gmail.com')
(592, '6910151867', 'Austen@gmail.com')
(593, '3749531459', 'Austin@gmail.com')
(594, '2062433655', 'Auston@gmail.com')
(598, '3399964133', 'Authur@gmail.com')
(599, '4266629877', 'Autry@gmail.com')
(601, '1886777907', 'Ava@gmail.com')
(608, '8903986929', 'Avon@gmail.com')
(609, '2307117687', 'Axel@gmail.com')
(611, '5481161708', 'Ayana@gmail.com')
(619, '4607310157', 'Azalee@gmail.com')
(620, '9761065963', 'Azaria@gmail.com')
(621, '7608601451', 'Azul@gmail.com')
(624, '8487141279', 'Babette@gmail.com')
(636, '8202851270', 'Barbara@gmail.com')
(637, '4816307359', 'Barbie@gmail.com')
(639, '4901512062', 'Barnard@gmail.com')
(645, '9759918761', 'Barrie@gmail.com')
(648, '8174368216', 'Bart@gmail.com')
(649, '9047754742', 'Bartholomew@gmail.com')
(660, '7070816871', 'Beatrice@gmail.com')
(664, '5098426291', 'Beaulah@gmail.com')
(674, '6818347605', 'Belen@gmail.com')
(686, '3303256850', 'Benita@gmail.com')
(687, '3161304916', 'Benito@gmail.com')
(689, '3304611608', 'Benjamen@g

### 19. List all retailer ids whose products, user_id = 1 have purchased.

In [115]:
 def list_retailers(mydb,user):
    
    query = """
    select Products.retailer_ID
    from Products inner join
    (select *
    from Orders
    where Orders.user_ID= %s ) as A
    on A.product_ID=Products.product_ID;

    """
    dbcursor=mydb.cursor()
    dbcursor.execute(query,(user,))
    print("Retailer Ids")
    for line in dbcursor:
        print(line)

In [116]:
list_retailers(mydb,1)

Retailer Ids
(19,)
(22,)
(48,)
(47,)
(47,)
(10,)
(48,)
(40,)
(10,)


### 20. Write a query to update the discount on all new products by 15% and store it as a new table holi_Deals. Note: Any product that is added to the database in the last 100 days is considered to be a new product.

In [185]:
def new_deals(mydb,tablename,discount):


    table_create ="""
    CREATE TABLE if not exists {table} (
    product_ID int,
    product_Name varchar(50),
    category_ID int,
    date_added date,
    price float(2),
    retailer_ID int);
    """.format(table=tablename)

    dbcursor = mydb.cursor()
    dbcursor.execute(table_create)
    mydb.commit()
    
    discount_factor = 1-int(discount)/100
    query="""

    INSERT INTO {table}
    (product_ID,product_Name,category_ID,date_added,price,retailer_ID) 
    SELECT product_ID,product_Name,category_ID,date_added,price*%s,retailer_ID 
    from Products 
    WHERE date_added >=(DATE(NOW()) - INTERVAL 100 DAY) 
    ORDER BY date_added DESC;


    """.format(table=tablename)

    dbcursor.execute(query,(discount_factor,))
    mydb.commit()
    dbcursor.execute("select * from {table}".format(table=tablename))
    for line in dbcursor:
        print(line)

In [186]:
new_deals(mydb,'holi_Deals','15'    )

(55, 'The Intelligent Investor', 3, datetime.date(2021, 2, 1), 95.2, 44)
(57, 'The Kite Runner', 3, datetime.date(2021, 1, 28), 255.0, 39)
(56, 'The lean Startup', 3, datetime.date(2021, 1, 24), 212.5, 40)
(58, 'The Mechanic', 3, datetime.date(2021, 1, 20), 340.0, 39)
(54, 'Cut and Sew Denim Jeans', 1, datetime.date(2020, 12, 13), 727.6, 49)
(53, 'Slim Fit Denim Jeans', 1, datetime.date(2020, 12, 12), 449.65, 48)
(48, 'Mens Bomber Jacket', 1, datetime.date(2020, 12, 10), 859.35, 48)
(55, 'The Intelligent Investor', 3, datetime.date(2021, 2, 1), 95.2, 44)
(57, 'The Kite Runner', 3, datetime.date(2021, 1, 28), 255.0, 39)
(56, 'The lean Startup', 3, datetime.date(2021, 1, 24), 212.5, 40)
(58, 'The Mechanic', 3, datetime.date(2021, 1, 20), 340.0, 39)
(54, 'Cut and Sew Denim Jeans', 1, datetime.date(2020, 12, 13), 727.6, 49)
(53, 'Slim Fit Denim Jeans', 1, datetime.date(2020, 12, 12), 449.65, 48)
(48, 'Mens Bomber Jacket', 1, datetime.date(2020, 12, 10), 859.35, 48)
(55, 'The Intelligent In